Considerar el cifrado por bloques MiniAES descrito en los apuntes

Apartado 1.- Calcula $E_{dni}(0x0134567)$ usando el modo CBC e IV=0x0001

In [1]:
F.<xi>=GF(2^4, modulus=GF(2)[x](x^4+x+1))
F.modulus()
def nibblesub(ll):
    xx = vector(ll)
    if xx != 0:
        xx=vector(reversed(vector(F(vector(reversed(xx)))^(-1))))
    return list(xx*matrix(GF(2),4,[0,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1])+vector([0,0,1,1]))

def shiftrow(ll):
    return [ll[0],ll[3],ll[2],ll[1]]

def mixcolumn(ll):
    return [list(reversed(vector(ele))) for ele in (matrix(F,2,[xi+1,xi,xi,xi+1])*matrix(F,2,[F(vector(reversed(ll[0]))), F(vector(reversed(ll[2]))), F(vector(reversed(ll[1]))), F(vector(reversed(ll[3])))])).transpose().list()]

def addroundkey(ll,kk):
    return [list(vector(ll[0])+vector(kk[0])), list(vector(ll[1])+vector(kk[1])), list(vector(ll[2])+vector(kk[2])), list(vector(ll[3])+vector(kk[3]))]

def key_squedule(kk):
    ww = list(reversed(vector(GF(2),kk.bits())))
    while len(ww)<16:
        ww = [GF(2)(0)]+ww
    ww = [[ww[0],ww[1],ww[2],ww[3]],[ww[4],ww[5],ww[6],ww[7]],[ww[8],ww[9],ww[10],ww[11]],[ww[12],ww[13],ww[14],ww[15]]]
    ww += [list(vector(ww[0])+vector(nibblesub(ww[3]))+vector(GF(2),[0,0,0,1]))]
    ww += [list(vector(ww[1]) + vector(ww[4]))]
    ww += [list(vector(ww[2]) + vector(ww[5]))]
    ww += [list(vector(ww[3]) + vector(ww[6]))]
    ww += [list(vector(ww[4])+vector(nibblesub(ww[7]))+vector(GF(2),[0,0,1,0]))]
    ww += [list(vector(ww[5]) + vector(ww[8]))]
    ww += [list(vector(ww[6]) + vector(ww[9]))]
    ww += [list(vector(ww[7]) + vector(ww[10]))]
    return ww

def MiniAES(kk,mm):
    ww = key_squedule(kk)
    estado = list(reversed(vector(GF(2),mm.bits())))
    while len(estado)< 16:
        estado = [GF(2)(0)]+estado
    estado = [[estado[0],estado[1],estado[2],estado[3]],[estado[4],estado[5],estado[6],estado[7]],[estado[8],estado[9],estado[10],estado[11]],[estado[12],estado[13],estado[14],estado[15]]]
    estado = addroundkey(estado,[ww[0],ww[1],ww[2],ww[3]])
    estado = [nibblesub(ele) for ele in estado]
    estado = shiftrow(estado)
    estado = mixcolumn(estado)
    estado = addroundkey(estado,[ww[4],ww[5],ww[6],ww[7]])
    estado = [nibblesub(ele) for ele in estado]
    estado = shiftrow(estado)
    estado = addroundkey(estado,[ww[8],ww[9],ww[10],ww[11]])
    salida = list(reversed(estado[0]+estado[1]+estado[2]+estado[3]))
    return estado,Integer(salida,base=2).hex()

In [2]:
clave=77021622%(65536)

In [3]:
mensaje=list(0x01234567.bits())
mensaje

[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1]

In [4]:
mens2=[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0]
mens1=[1, 1, 0, 0, 0, 1, 0, 0, 1]+[0,0,0,0,0,0,0]

In [5]:
c0='0x0001'

In [6]:
c1='0x'+MiniAES(clave, Integer(mens1, base=2)^^(Integer(c0)))[1]
c1

'0x70ff'

In [7]:
c2='0x'+MiniAES(clave, Integer(mens2, base=2)^^(Integer(c1)))[1]
c2

'0x2599'

Luego $E_{dni}(0x0134567)=c0||c1||c2=0x000170ff2599$

Apartado 2

Calcula $E_{dni}(0x0134567)$ usando el modo CFB, r=11, IV=0x0001

In [8]:
clave=77021622%(65536)

In [9]:
def msb(num,r):
    lista=num.bits()
    while len(lista)<16:
        lista=lista+[0]
    return lista[-r:]
def lsb(num,r):
    lista=num.bits()
    while len(lista)<16:
        lista=lista+[0]
    return lista[:r]

In [10]:
mens=0x01234567.bits()
mens

[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1]

In [11]:
mens=mens+[0,0,0,0,0,0,0,0]

In [12]:
mens1=[0,0,1,0,0,0,0,0,0,0,0]
mens2=[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1]
mens3=[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1]

In [13]:
x1='0x0001'
Integer(x1).bits()
x1=[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [15]:
c1='0x'+(Integer(mens1, base=2)^^(Integer(msb(Integer('0x'+MiniAES(clave, Integer((x1),base=2))[1]),11), base=2))).hex()
print (c1)
Integer(c1).bits()

0x776


[0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]

In [16]:
x2=list(reversed(list(reversed(lsb(Integer(x1,base=2),5)))+list(reversed(Integer(c1).bits()))))
x2

[0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0]

In [18]:
c2='0x'+(Integer(mens2, base=2)^^(Integer(msb(Integer('0x'+MiniAES(clave, Integer((x2),base=2))[1]),11), base=2))).hex()
print (c2)
Integer(c2).bits()

0x6c1


[1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]

In [19]:
x3=list(reversed(list(reversed(lsb(Integer(x2,base=2),5)))+list(reversed(Integer(c2).bits()))))
x3

[1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]

In [21]:
c3='0x'+(Integer(mens3, base=2)^^(Integer(msb(Integer('0x'+MiniAES(clave, Integer((x3),base=2))[1]),11), base=2))).hex()
print (c3)
Integer(c3).bits()

0x37b


[1, 1, 0, 1, 1, 1, 1, 0, 1, 1]

Luego $E_{dni}(0x0134567)=c1||c2||c3=0x7766c137b$